
# ATAR Algorithm with MNE RAW Object

In this example, we show, how to apply ATAR algorithm with MNE Raw object.

We will read EDF File as MNE Raw file then extract EEG Data from it, scale and transport it for ATAR.
After ATAR, we will put EEG Data back to MNE Raw object. This is perticullary useful, when all the other analysis is done in MNE Framework.

We do not show the analysis part, only, **how to extract EEG from MNE Raw, apply ATAR and put it back**

Same can be applied, when EEG Data file is 

    * FIF
    * EDF
    * BDF
    * EGI
    * vhdr
    
Any file that can be read by `mne.io.read_raw**` can be used in this way to apply ATAR.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, re, time, glob, requests
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


import spkit as sp
print('spkit-version : ',sp.__version__)

import mne

## Download sample file



In [ ]:
# Download file, if not already downloaded


path ='https://github.com/spkit/data_samples/raw/main/files/Resting_EPOCX_14Ch_Sample1.edf'


file_name = 'Resting_EPOCX_14Ch_Sample1.edf'

if not(os.path.exists(file_name)):
    req = requests.get(path)
    with open(file_name, 'wb') as f:
            f.write(req.content)

## Read EDF File  with MNE Raw



In [ ]:
raw = mne.io.read_raw_edf(file_name,preload=True)
print(raw.info)
fs = raw.info["sfreq"]

print('Channel Names')
sp.utils.pretty_print(raw.ch_names)

eeg_ch  =['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
ch_names = list(raw.ch_names)
ch_type = ['eeg' if ch in eeg_ch else 'bio' for ch in ch_names]
raw.set_channel_types(dict(zip(ch_names, ch_type)))

raw.filter(l_freq=0.5,h_freq=None, picks='eeg')

fig  = raw.copy().pick('eeg').plot(scalings=dict(eeg=100e-6), duration=20, start=10)

## Extract EEG and Apply ATAR



In [ ]:
X = raw.get_data()

#EEG Channels
XE = X[6:20].copy()
print(XE.shape, XE.min(), XE.max())

# Scale in uV (order of 100s) and transpose (channle at axis =1)
XE = XE.T*1e6

# ATAR
XR = sp.eeg.ATAR(XE,wv='db3',winsize=128, thr_method='ipr', OptMode='elim',beta=0.1,k2=100,verbose=1,use_joblib=False)

# Scale and transpose back
XR = XR.T*1e-6

## Putting back EEG to Original Raw MNE



In [ ]:
raw1 = raw.copy()
Xi = raw1.get_data()
Xi[6:20] = XR
raw1._data = Xi

## Plot for comparison 



In [ ]:
fig  = raw1.copy().pick('eeg').plot(scalings=dict(eeg=100e-6), duration=20, start=10)